# LangGrpah, minimal code

🤖[LangGraph](https://python.langchain.com/docs/langgraph) is a library for building stateful, multi-actor applications with LLMs, built on top of (and intended to be used with) LangChain.

This code is the first step of using LangGraph to bulid generic and simple graphs even without LLMs.

By: [Ibrahim Sobh](https://www.linkedin.com/in/ibrahim-sobh-phd-8681757/)



## Install

In [ ]:
%pip install -U --quiet  langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 2.1 MB/s eta 0:00:00


In [ ]:
import random
import string

In [ ]:
from typing import Dict, TypedDict
from langchain_core.messages import BaseMessage


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

## Guess the number, if fail, try again



In [ ]:
def generate_number(state):
    """
    Generate a random number between 1 and 10

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """

    ## State
    state_dict = state["keys"]
    iter = state_dict["iterations"]

    generated_solution = random.randint(0, 10)
    iter = iter+1
    return {"keys": {"generation": generated_solution, "iterations":iter}}

In [ ]:
def check_number(state):
    """
    Check code block execution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    ## State
    state_dict = state["keys"]
    generated_solution = state_dict["generation"]
    iter = state_dict["iterations"]

    if generated_solution != 7:  # my magic number
      error = "FAIL"
    else:
      error = "None"

    return {"keys": {"generation": generated_solution,
                     "error": error,
                     "iterations":iter}}

In [ ]:
def decide_to_finish(state):
    """
    Determines whether to finish or re-try

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    state_dict = state["keys"]
    generated_solution = state_dict["generation"]
    error = state_dict["error"]
    iter = state_dict["iterations"]

    print(state)
    if error == "None":
        return "end"
    else:
        return "generate"

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate_number", generate_number)  # generation a number
workflow.add_node("check_number", check_number)  # check number

# Build graph
workflow.set_entry_point("generate_number")
workflow.add_edge("generate_number", "check_number")
workflow.add_conditional_edges(
    "check_number",
    decide_to_finish,
    {
        "end": END,
        "generate": "generate_number",
    },
)

# Compile
app = workflow.compile()

In [ ]:
config = {"recursion_limit": 200}
fina_state = app.invoke({"keys":{"iterations":0}},config=config)

{'keys': {'generation': 9, 'error': 'FAIL', 'iterations': 1}}
{'keys': {'generation': 1, 'error': 'FAIL', 'iterations': 2}}
{'keys': {'generation': 2, 'error': 'FAIL', 'iterations': 3}}
{'keys': {'generation': 3, 'error': 'FAIL', 'iterations': 4}}
{'keys': {'generation': 10, 'error': 'FAIL', 'iterations': 5}}
{'keys': {'generation': 8, 'error': 'FAIL', 'iterations': 6}}
{'keys': {'generation': 7, 'error': 'None', 'iterations': 7}}


## Dynamic Graph

Try to guess a number or char, if you fail try again.

In [ ]:
def generate_char(state):
    """
    Generate a random number between 1 and 100

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """

    ## State
    state_dict = state["keys"]
    iter = state_dict["iterations"]

    generated_solution = random.choice(string.ascii_letters)
    iter = iter+1
    return {"keys": {"generation": generated_solution, "iterations":iter}}

In [ ]:
def check_char(state):
    """
    Check code block execution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    ## State
    state_dict = state["keys"]
    generated_solution = state_dict["generation"]
    iter = state_dict["iterations"]

    if generated_solution in "IBRAHIM SOBH ibrahim sobh":  # my magic char
      error = "None"
    else:
      error = "FAIL"

    return {"keys": {"generation": generated_solution,
                     "error": error,
                     "iterations":iter}}

In [ ]:
workflow = StateGraph(GraphState)

# Configurable Graph
# Selec the typr of generation: char or number
my_configuration = "char"
my_configuration = "number"

if my_configuration == "number": # gusse the number
  # add nodes
  workflow.add_node("generate_number", generate_number)  # generation a number
  workflow.add_node("check_number", check_number)  # check number
  # add edge
  workflow.set_entry_point("generate_number")
  workflow.add_edge("generate_number", "check_number")
  workflow.add_conditional_edges(
      "check_number",
      decide_to_finish,
      {
          "end": END,
          "generate": "generate_number",
      },)

if my_configuration == "char": # gusse the number
  # add nodes
  workflow.add_node("generate_char", generate_char)  # generation a char
  workflow.add_node("check_char", check_char)  # check char
  # add edge
  workflow.set_entry_point("generate_char")
  workflow.add_edge("generate_char", "check_char")
  workflow.add_conditional_edges(
      "check_char",
      decide_to_finish,
      {
          "end": END,
          "generate": "generate_char",
      },)


# Compile
app = workflow.compile()

In [ ]:
config = {"recursion_limit": 200}
final_state = app.invoke({"keys":{"iterations":0}},config=config)

{'keys': {'generation': 10, 'error': 'FAIL', 'iterations': 1}}
{'keys': {'generation': 2, 'error': 'FAIL', 'iterations': 2}}
{'keys': {'generation': 3, 'error': 'FAIL', 'iterations': 3}}
{'keys': {'generation': 4, 'error': 'FAIL', 'iterations': 4}}
{'keys': {'generation': 10, 'error': 'FAIL', 'iterations': 5}}
{'keys': {'generation': 9, 'error': 'FAIL', 'iterations': 6}}
{'keys': {'generation': 10, 'error': 'FAIL', 'iterations': 7}}
{'keys': {'generation': 4, 'error': 'FAIL', 'iterations': 8}}
{'keys': {'generation': 5, 'error': 'FAIL', 'iterations': 9}}
{'keys': {'generation': 4, 'error': 'FAIL', 'iterations': 10}}
{'keys': {'generation': 8, 'error': 'FAIL', 'iterations': 11}}
{'keys': {'generation': 1, 'error': 'FAIL', 'iterations': 12}}
{'keys': {'generation': 6, 'error': 'FAIL', 'iterations': 13}}
{'keys': {'generation': 3, 'error': 'FAIL', 'iterations': 14}}
{'keys': {'generation': 9, 'error': 'FAIL', 'iterations': 15}}
{'keys': {'generation': 5, 'error': 'FAIL', 'iterations': 16}

## More Complex Graph

Try to guess a number, if succeed, try to guess a char, if fail in number or chat, try again.  






In [ ]:
workflow = StateGraph(GraphState)


# add nodes
workflow.add_node("generate_number", generate_number)  # generation a number
workflow.add_node("check_number", check_number)  # check number
workflow.add_node("generate_char", generate_char)  # generation a char
workflow.add_node("check_char", check_char)  # check char

workflow.set_entry_point("generate_number")
workflow.add_edge("generate_number", "check_number")
workflow.add_edge("generate_char", "check_char")

workflow.add_conditional_edges(
    "check_number",
    decide_to_finish,
    {
        "end": "generate_char",
        "generate": "generate_number",
    },)

workflow.add_conditional_edges(
      "check_char",
      decide_to_finish,
      {
          "end": END,
          "generate": "generate_char",
      },)


# Compile
app = workflow.compile()

In [ ]:
config = {"recursion_limit": 200}
app.invoke({"keys":{"iterations":0}},config=config)

{'keys': {'generation': 3, 'error': 'FAIL', 'iterations': 1}}
{'keys': {'generation': 5, 'error': 'FAIL', 'iterations': 2}}
{'keys': {'generation': 7, 'error': 'None', 'iterations': 3}}
{'keys': {'generation': 'h', 'error': 'None', 'iterations': 4}}


{'keys': {'generation': 'h', 'error': 'None', 'iterations': 4}}